# Информация о покупателях с добавлением меток классов (olist_classified_public_dataset.csv)

In [190]:
import pandas as pd
import re

In [191]:
classified_data = pd.read_csv(r'C:\Users\Вячеслав\Documents\olist_churn\olist-ml-churn-prediction\data\raw\olist_classified_public_dataset.csv').drop('Unnamed: 0', axis=1)

In [192]:
display(classified_data.head())
print(classified_data.shape)
print(classified_data.dtypes.nunique())

,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01.000000,2017-08-30 11:55:08.970352,2017-09-21 00:00:00.000000,2017-09-08 20:35:27.276847,...,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido
1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36.000000,2017-09-26 09:28:10.922048,2017-10-24 00:00:00.000000,2017-09-29 21:13:04.984841,...,0,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido
2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42.000000,2018-01-17 07:29:56.419769,2018-02-05 00:00:00.000000,2018-01-23 17:51:31.134866,...,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega
3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42.000000,2018-02-06 05:31:50.990164,2018-03-13 00:00:00.000000,2018-02-20 19:38:06.633080,...,3,0,0,0,0,0,0,0,atrasado,problemas_de_entrega
4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42.000000,2017-12-08 02:36:49.587515,2018-01-03 00:00:00.000000,2017-12-19 22:33:18.952512,...,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega


(3584, 33)
3


In [193]:
print(classified_data.dtypes)

id                                 int64
order_status                      object
order_products_value             float64
order_freight_value              float64
order_items_qty                    int64
order_sellers_qty                  int64
order_purchase_timestamp          object
order_aproved_at                  object
order_estimated_delivery_date     object
order_delivered_customer_date     object
customer_city                     object
customer_state                    object
customer_zip_code_prefix           int64
product_category_name             object
product_name_lenght                int64
product_description_lenght         int64
product_photos_qty                 int64
review_score                       int64
review_comment_title             float64
review_comment_message            object
review_creation_date              object
review_answer_timestamp           object
votes_before_estimate              int64
votes_delayed                      int64
votes_low_qualit

#### Типы данных в порядке за исключением временных признаков. Сконвертируем их в datetime позже.

#### Разделим признаки на 3 группы: категориальные, численные (+ дискретные) и временные.

In [194]:
categorical_features = ['order_status', 'customer_city', 'customer_state',
                        'product_category_name', 'most_voted_subclass', 'most_voted_class']
numerical_features = ['order_products_value', 'order_freight_value', 'order_items_qty',
                      'order_sellers_qty', 'customer_zip_code_prefix', 'product_name_lenght',
                      'product_description_lenght', 'product_photos_qty', 'review_score',
                      'review_comment_title', 'votes_before_estimate', 'votes_delayed',
                      'votes_low_quality', 'votes_return', 'votes_not_as_anounced',
                      'votes_partial_delivery', 'votes_other_delivery', 'votes_other_order',
                      'votes_satisfied']
time_features = ['order_purchase_timestamp', 'order_aproved_at', 'order_estimated_delivery_date',
                 'order_delivered_customer_date', 'review_creation_date', 'review_answer_timestamp']
other_features = ['id', 'review_comment_message']

In [195]:
print(classified_data.isna().sum())

id                                  0
order_status                        0
order_products_value                0
order_freight_value                 0
order_items_qty                     0
order_sellers_qty                   0
order_purchase_timestamp            0
order_aproved_at                    0
order_estimated_delivery_date       0
order_delivered_customer_date     117
customer_city                       0
customer_state                      0
customer_zip_code_prefix            0
product_category_name               0
product_name_lenght                 0
product_description_lenght          0
product_photos_qty                  0
review_score                        0
review_comment_title             3584
review_comment_message              0
review_creation_date                0
review_answer_timestamp             0
votes_before_estimate               0
votes_delayed                       0
votes_low_quality                   0
votes_return                        0
votes_not_as

#### В признаке *review_comment_title* пропущены все значения. Уберем его.

In [196]:
classified_data = classified_data.drop('review_comment_title', axis=1)

## Категориальные признаки

In [197]:
display(classified_data[categorical_features].head())

,order_status,customer_city,customer_state,product_category_name,most_voted_subclass,most_voted_class
0,delivered,Belo Horizonte,MG,beleza_saude,satisfeito,satisfeito_com_pedido
1,delivered,Pocos de Caldas,MG,brinquedos,antes_prazo,satisfeito_com_pedido
2,delivered,Sao Jose dos Campos,SP,ferramentas_jardim,entrega_parcial,problemas_de_entrega
3,delivered,Ribeirao Preto,SP,informatica_acessorios,atrasado,problemas_de_entrega
4,delivered,RIO DE JANEIRO,RJ,cama_mesa_banho,entrega_parcial,problemas_de_entrega


In [198]:
for col in categorical_features:
    print(f"--- {col} ---")
    print(classified_data[col].value_counts())
    print()

--- order_status ---
order_status
delivered     3467
shipped         48
canceled        25
invoiced        24
processing      20
Name: count, dtype: int64

--- customer_city ---
customer_city
Sao Paulo         438
RIO DE JANEIRO    276
Belo Horizonte     79
BRASILIA           74
Salvador           65
                 ... 
Buriticupu          1
Camacari            1
Toledo              1
Roda Velha          1
Braganca            1
Name: count, Length: 897, dtype: int64

--- customer_state ---
customer_state
SP    1369
RJ     505
MG     438
RS     188
PR     161
BA     150
SC     118
GO      84
PE      76
DF      74
ES      70
CE      58
PA      58
MT      42
MA      41
AL      24
PI      24
PB      23
RN      21
MS      18
SE      16
TO      10
RO       7
AM       5
AC       3
RR       1
Name: count, dtype: int64

--- product_category_name ---
product_category_name
cama_mesa_banho                                   471
moveis_decoracao                                  302
beleza_saude   

#### Распределения значений категориальных признаков достаточно чистые. Однако нужно совершить дополнительные преобразования:
1. Привести значения строк всех буквенных признаков, кроме *review_comment_message* к lowercase с отступами (пример: RIO DE JANEIRO --> rio_de_janeiro).
2. С помощью набора данных product_category_name_translation.csv перевести названия товаров с бразильского на английский для лучшей интерпретируемости.

#### Приведение к lowercase:

In [199]:
def to_new_value_fmt(s: str) -> str:
    if pd.isna(s): # пропуски не трогаем
        return s
    txt = str(s).lower().strip()
    txt = re.sub(r"[ \t\.\-]+", "_", txt)
    txt = re.sub(r"[^a-z0-9_]", "", txt)
    return txt

In [200]:
classified_data[categorical_features] = classified_data[categorical_features].map(to_new_value_fmt)

In [201]:
for col in categorical_features:
    print(f"--- {col} ---")
    print(classified_data[col].value_counts())
    print()

--- order_status ---
order_status
delivered     3467
shipped         48
canceled        25
invoiced        24
processing      20
Name: count, dtype: int64

--- customer_city ---
customer_city
sao_paulo               478
rio_de_janeiro          276
belo_horizonte           79
brasilia                 74
salvador                 65
                       ... 
ribeirao_cascalheira      1
visconde_de_mau           1
borborema                 1
rodeiro                   1
braganca                  1
Name: count, Length: 896, dtype: int64

--- customer_state ---
customer_state
sp    1369
rj     505
mg     438
rs     188
pr     161
ba     150
sc     118
go      84
pe      76
df      74
es      70
ce      58
pa      58
mt      42
ma      41
al      24
pi      24
pb      23
rn      21
ms      18
se      16
to      10
ro       7
am       5
ac       3
rr       1
Name: count, dtype: int64

--- product_category_name ---
product_category_name
cama_mesa_banho                                   471
mov

#### Как можно заметить, некоторые одинаковые города с разным написанием стали одного типа (например для sao_paulo было 438, стало 478). Однако это также может означать, что были города с одинаковыми названиями, но фактически разными. Проверим данное предположение:

In [202]:
city_state_counts = (
    classified_data
    .groupby('customer_city')['customer_state']
    .nunique()
)

conflict_cities = city_state_counts[city_state_counts > 1]

print(f"Количество «проблемных» городов: {len(conflict_cities)}")

Количество «проблемных» городов: 6


#### Действительно, такие города есть. Посмотрим внимательнее:

In [203]:
# все строки, где customer_city входит в список «конфликтных»
conflicts_df = (classified_data
                .loc[classified_data['customer_city'].isin(conflict_cities.index),
                        ['customer_city', 'customer_state']]
                .drop_duplicates()
                .sort_values(['customer_city', 'customer_state']))
print(conflicts_df)

     customer_city customer_state
1101      alvorada             rs
1866      alvorada             to
1826  bandeirantes             ms
447   bandeirantes             pr
86    campo_grande             al
316   campo_grande             ms
2307       itapeva             mg
844        itapeva             sp
104     sao_carlos             sc
132     sao_carlos             sp
807        valenca             ba
3431       valenca             rj


#### Простым решением я вижу вручную переименовать эти города, так как их немного. Переименуем в формат: alvorada и alvorada_2:

In [204]:
mask = ((classified_data['customer_city']  == 'alvorada') & (classified_data['customer_state'] == 'rs'))
classified_data.loc[mask, 'customer_city'] = 'alvorada_2'

mask = ((classified_data['customer_city']  == 'bandeirantes') & (classified_data['customer_state'] == 'ms'))
classified_data.loc[mask, 'customer_city'] = 'bandeirantes_2'

mask = ((classified_data['customer_city']  == 'campo_grande') & (classified_data['customer_state'] == 'al'))
classified_data.loc[mask, 'customer_city'] = 'campo_grande_2'

mask = ((classified_data['customer_city']  == 'itapeva') & (classified_data['customer_state'] == 'mg'))
classified_data.loc[mask, 'customer_city'] = 'itapeva_2'

mask = ((classified_data['customer_city']  == 'sao_carlos') & (classified_data['customer_state'] == 'sc'))
classified_data.loc[mask, 'customer_city'] = 'sao_carlos_2'

mask = ((classified_data['customer_city']  == 'valenca') & (classified_data['customer_state'] == 'ba'))
classified_data.loc[mask, 'customer_city'] = 'valenca_2'

#### Теперь переведем названия товаров. Загрузим датасет с английскими названиями:

In [205]:
translation = pd.read_csv(r'C:\Users\Вячеслав\Documents\olist_churn\olist-ml-churn-prediction\data\raw\product_category_name_translation.csv')

In [206]:
display(translation.head())

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


#### Используем left join, где левый датасет - classified_data.

In [207]:
classified_data = classified_data.merge(translation, how='left', on='product_category_name')

In [208]:
classified_data = classified_data.drop('product_category_name', axis=1)

In [209]:
display(classified_data.head())

,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01.000000,2017-08-30 11:55:08.970352,2017-09-21 00:00:00.000000,2017-09-08 20:35:27.276847,...,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,health_beauty
1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36.000000,2017-09-26 09:28:10.922048,2017-10-24 00:00:00.000000,2017-09-29 21:13:04.984841,...,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido,toys
2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42.000000,2018-01-17 07:29:56.419769,2018-02-05 00:00:00.000000,2018-01-23 17:51:31.134866,...,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,garden_tools
3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42.000000,2018-02-06 05:31:50.990164,2018-03-13 00:00:00.000000,2018-02-20 19:38:06.633080,...,0,0,0,0,0,0,0,atrasado,problemas_de_entrega,computers_accessories
4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42.000000,2017-12-08 02:36:49.587515,2018-01-03 00:00:00.000000,2017-12-19 22:33:18.952512,...,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,bed_bath_table


## Временные признаки

#### Теперь обработаем time-фичи:

In [210]:
display(classified_data[time_features].head())

,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,review_creation_date,review_answer_timestamp
0,2017-08-30 11:41:01.000000,2017-08-30 11:55:08.970352,2017-09-21 00:00:00.000000,2017-09-08 20:35:27.276847,2017-09-09 00:00:00.000000,2017-09-10 03:27:54+00:00
1,2017-09-26 09:13:36.000000,2017-09-26 09:28:10.922048,2017-10-24 00:00:00.000000,2017-09-29 21:13:04.984841,2017-09-30 00:00:00.000000,2017-10-03 05:34:20+00:00
2,2018-01-15 15:50:42.000000,2018-01-17 07:29:56.419769,2018-02-05 00:00:00.000000,2018-01-23 17:51:31.134866,2018-01-24 00:00:00.000000,2018-02-02 17:42:43+00:00
3,2018-02-04 11:16:42.000000,2018-02-06 05:31:50.990164,2018-03-13 00:00:00.000000,2018-02-20 19:38:06.633080,2018-02-21 00:00:00.000000,2018-02-22 02:09:12+00:00
4,2017-12-07 11:58:42.000000,2017-12-08 02:36:49.587515,2018-01-03 00:00:00.000000,2017-12-19 22:33:18.952512,2017-12-20 00:00:00.000000,2017-12-23 04:17:03+00:00


#### Мы уже знаем о пропусках в столбце order_delivered_customer_date. Они означают, что заказ пользователю еще не был доставлен. Было решено убрать этих пользователей из анализа, так как мы не имеем информации об оценках качества продукта.

In [211]:
classified_data.shape

(3584, 32)

In [212]:
classified_data = classified_data.dropna(subset=['order_delivered_customer_date'])

In [213]:
classified_data.shape

(3467, 32)

#### Переведем тип каждого временного признака в datetime из object:

In [225]:
for col in classified_data[time_features].drop('review_answer_timestamp', axis=1):
    classified_data[col] = pd.to_datetime(
        classified_data[col],               # убираем лишние пробелы
        format='%Y-%m-%d %H:%M:%S',                  # если микросекунды бывают не всегда — см. вариант 2
        errors='coerce'                                 # ошибки → NaT
    )
#classified_data['review_answer_timestamp'] = pd.to_datetime(classified_data['review_answer_timestamp'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')

In [237]:
classified_data['review_answer_timestamp'] = pd.to_datetime(
    classified_data['review_answer_timestamp'],
    utc=True,           # раз везде +0000, сразу локализуем в UTC
    errors='raise')      # если встретите битую строку — получите исключение
classified_data['review_answer_timestamp'] = classified_data['review_answer_timestamp'].dt.tz_localize(None)

In [238]:
display(classified_data[time_features])

,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,review_creation_date,review_answer_timestamp
0,2017-08-30 11:41:01,2017-08-30 11:55:08.970352,2017-09-21,2017-09-08 20:35:27.276847,2017-09-09,2017-09-10 03:27:54
1,2017-09-26 09:13:36,2017-09-26 09:28:10.922048,2017-10-24,2017-09-29 21:13:04.984841,2017-09-30,2017-10-03 05:34:20
2,2018-01-15 15:50:42,2018-01-17 07:29:56.419769,2018-02-05,2018-01-23 17:51:31.134866,2018-01-24,2018-02-02 17:42:43
3,2018-02-04 11:16:42,2018-02-06 05:31:50.990164,2018-03-13,2018-02-20 19:38:06.633080,2018-02-21,2018-02-22 02:09:12
4,2017-12-07 11:58:42,2017-12-08 02:36:49.587515,2018-01-03,2017-12-19 22:33:18.952512,2017-12-20,2017-12-23 04:17:03
...,...,...,...,...,...,...
3578,2018-02-12 16:07:34,2018-02-12 16:27:39.816713,2018-02-26,2018-02-15 19:16:54.139873,2018-02-16,2018-02-20 13:52:22
3580,2017-05-11 12:29:55,2017-05-11 13:05:49.204273,2017-05-31,2017-05-16 14:17:47.915865,2017-05-17,2017-05-18 04:15:26
3581,2017-05-21 20:15:25,2017-05-23 03:42:32.806938,2017-06-13,2017-05-30 14:14:50.522442,2017-05-31,2017-06-01 21:36:36
3582,2017-09-29 16:05:14,2017-09-29 16:27:57.145051,2017-11-13,2017-10-25 03:35:57.037048,2017-10-25,2017-10-25 21:20:43


In [230]:
print(classified_data[time_features].dtypes)

order_purchase_timestamp              datetime64[ns]
order_aproved_at                      datetime64[ns]
order_estimated_delivery_date         datetime64[ns]
order_delivered_customer_date         datetime64[ns]
review_creation_date                  datetime64[ns]
review_answer_timestamp          datetime64[ns, UTC]
dtype: object


#### На основе данных временных признаков можно создать новые полезные и хорошо интерпретируемые:
1. Время между оплатой заказа и подтверждением заказа на сайте (order_approved_at - order_purchase_timestamp).
2. Время между подтверждением заказа и ориентировочным времени доставки (order_estimated_delivery_date - order_aproved_at).
3. Время между фактическим времени доставки и ожидаемым (order_delivered_customer_date - order_estimated_delivery_date).
4. Время между рассылкой с просьбой оценить качество и ответом покупателя (review_answer_timestamp - review_creation_date).
5. Время между подтверждением заказа и его прибытием (order_delivered_customer_date - order_aproved_at).

In [247]:
def new_time_features(data, feature_1, feature_2):
    return (data[feature_2] - data[feature_1]).dt.total_seconds() / 3600

In [257]:
classified_data['aproved-purchased'] = new_time_features(classified_data, 'order_purchase_timestamp', 'order_aproved_at')
classified_data['estimated_delivery-aproved'] = new_time_features(classified_data, 'order_aproved_at', 'order_estimated_delivery_date')
classified_data['actual_delivery-estimated_delivery'] = new_time_features(classified_data, 'order_estimated_delivery_date', 'order_delivered_customer_date')
classified_data['review_answer-review_creation'] = new_time_features(classified_data, 'review_creation_date', 'review_answer_timestamp')
classified_data['actual_delivery-aproved'] = new_time_features(classified_data, 'order_aproved_at', 'order_delivered_customer_date')

In [258]:
print(classified_data[['aproved-purchased', 'estimated_delivery-aproved', 'actual_delivery-estimated_delivery', 'review_answer-review_creation', 'actual_delivery-aproved']])

      aproved-purchased  estimated_delivery-aproved  \
0              0.235547                  516.080842   
1              0.243034                  662.530299   
2             39.654005                  448.500995   
3             42.252497                  834.469169   
4             14.635441                  621.386226   
...                 ...                         ...   
3578           0.334949                  319.538940   
3580           0.598390                  466.902999   
3581          31.452169                  500.290887   
3582           0.378651                 1063.534126   
3583           0.213226                  256.258996   

      actual_delivery-estimated_delivery  review_answer-review_creation  \
0                            -291.409090                      27.465000   
1                            -578.781949                      77.572222   
2                            -294.141351                     233.711944   
3                            -484.36482

#### Значения показывают количество часов разницы. В случае с разницей в доставке число отрицательное если заказ был доставлен раньше ориентировочного числа (скорее хорошо), и положительное, если была задержка по сравнению с ориентировочным числом (скорее плохо).

## Гипотеза 1: покупателю не нравится доставка ранее предполагаемого числа.

# Численные признаки